In [ ]:
import pandas as pd

def df_newlabel(df):
  df2 = pd.melt(df, id_vars = df.columns[~df.columns.str.contains('competency')])

  #역량명 숫자로 변경
  df2['variable'] = [int(var[-1]) for var in df2['variable']]
  df2.rename(columns = {'variable': 'label', 'value': 'score'}, inplace = True)

  return df2

def df_preprocessing(df, column_list : list):
  df1 = df.loc[:, column_list]

  #특수문자 및 불필요한 공백제거
  df1['student_assessment'] = df1['student_assessment'].str.replace("[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
  df1['student_assessment'] = df1['student_assessment'].str.replace(" +"," ")

  add_column = ['\t'.join(list(map(str, df1.loc[x, column_list].tolist()))) for x in range(len(df1))]
  score_list = df['score'] - 1

  data = [[add_column[i],score_list[i]] for i in range(len(df1))]


  return data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_3points_subject_concepts.csv")

#사용할 칼럼명 리스트로 지정
column_list = ['label', 'program_category', 'mission_category', 'student_assessment']

#사용할 역량번호 지정
target_number = 1

df2 = df_newlabel(df)
dataset = df_preprocessing(df2, column_list)

# 사용할 dataset
dataset

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

# https://blog.naver.com/newyearchive/223097878715

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
from imblearn.over_sampling import SMOTE

class BERTOverDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

        self.sentences1 = [self.sentences[i][0] for i in range(len(self.sentences))]

        smote = SMOTE(random_state = 2022)
        self.sen1_over, self.labels_over = smote.fit_resample(self.sentences1, self.labels)

        self.sen1_over = [np.array(sen) for sen in self.sen1_over]
        self.sen2_over = [sum(sen != 1) for sen in self.sen1_over]
        self.sen3_over = [np.zeros(max_len) for _ in self.sen1_over]

        self.sen_over = [(self.sen1_over[i], np.array(self.sen2_over[i]), self.sen3_over[i]) for i in range(len(self.sen1_over))]

    def __getitem__(self, i):
      return (self.sen_over[i] + (self.labels_over[i], ))

    def __len__(self):
      return (len(self.labels_over))

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

In [ ]:
# Setting parameters
max_len = 256
batch_size = 40
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, shuffle=True, random_state=34)

In [ ]:
data_train = BERTOverDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_history=[]
test_history=[]
train_loss_history=[]
test_loss_history=[]

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train_history.append(train_acc / (batch_id+1))
    train_loss_history.append(loss.data.cpu().numpy())

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))
    test_loss_history.append(loss.data.cpu().numpy())

<ipython-input-19-25e0bfef111a>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1189 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.2036504745483398 train acc 0.275
epoch 1 batch id 201 loss 0.9596861600875854 train acc 0.41293532338308453
epoch 1 batch id 401 loss 0.9353935122489929 train acc 0.4930798004987531
epoch 1 batch id 601 loss 0.755039393901825 train acc 0.5271214642262895
epoch 1 batch id 801 loss 0.7166288495063782 train acc 0.5612671660424466
epoch 1 batch id 1001 loss 0.6408514976501465 train acc 0.5860639360639364
epoch 1 train acc 0.6034582882774645


<ipython-input-19-25e0bfef111a>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/155 [00:00<?, ?it/s]

...

epoch 49 test acc 0.7820430107526877


  0%|          | 0/1189 [00:00<?, ?it/s]

epoch 50 batch id 1 loss 3.32765412167646e-05 train acc 1.0
epoch 50 batch id 201 loss 0.019973253831267357 train acc 0.9982587064676615
epoch 50 batch id 401 loss 3.6748482671100646e-05 train acc 0.9981296758104743
epoch 50 batch id 601 loss 3.738012310350314e-05 train acc 0.9982113144758744
epoch 50 batch id 801 loss 3.968380769947544e-05 train acc 0.9982521847690398
epoch 50 batch id 1001 loss 4.136448114877567e-05 train acc 0.9983516483516494
epoch 50 train acc 0.9983599663582846


  0%|          | 0/155 [00:00<?, ?it/s]

epoch 50 test acc 0.782043010752688


In [ ]:
train_history

[0.6034582882774645,
 0.7160969602306854,
 0.750598742440626,
 0.7672894389042422,
 0.7806239737274215,
 0.8040199847811296,
 0.8323841563538783,
 0.8621590772557974,
 0.8905632984901265,
 0.9192638872201516,
 0.933877007489288,
 0.9472475870078921,
 0.9553886819656427,
 0.9597560975609801,
 0.9654962153069846,
 0.9678531378909904,
 0.9731917577796502,
 0.975567703952905,
 0.9784272497897428,
 0.9805508830950412,
 0.982590412111022,
 0.9855550883095072,
 0.9855130361648468,
 0.9880992430613987,
 0.9882063759061253,
 0.9908536585365869,
 0.9924306139613136,
 0.9918839360807425,
 0.9945332211942814,
 0.9937762825904128,
 0.9945962994112704,
 0.9954183187152076,
 0.9961942809083266,
 0.9956896551724141,
 0.9964465937762836,
 0.9969512195121957,
 0.9971614802354924,
 0.9972245584524815,
 0.9974348191757786,
 0.9980866274179986,
 0.9977712363330535,
 0.9978763666947017,
 0.9983809924306142,
 0.9982968881412949,
 0.9980235492010093,
 0.9979604709840202,
 0.9982968881412954,
 0.99833894028595

In [ ]:
test_history

[0.6941397849462366,
 0.7483333333333335,
 0.7739784946236556,
 0.7771505376344089,
 0.7868817204301076,
 0.7781720430107526,
 0.7725806451612903,
 0.7796236559139786,
 0.759408602150538,
 0.7813978494623657,
 0.7394623655913981,
 0.772311827956989,
 0.7546236559139783,
 0.7771505376344089,
 0.7675268817204303,
 0.7610215053763442,
 0.7690860215053762,
 0.7680107526881718,
 0.7584946236559139,
 0.7715591397849458,
 0.772741935483871,
 0.766989247311828,
 0.7693548387096778,
 0.7612365591397849,
 0.7718817204301072,
 0.7598924731182792,
 0.7701075268817204,
 0.7745698924731185,
 0.7775268817204299,
 0.7736021505376344,
 0.7733333333333338,
 0.7722580645161291,
 0.7761827956989249,
 0.7750537634408603,
 0.7756451612903229,
 0.7780107526881721,
 0.7731182795698922,
 0.7752150537634409,
 0.7817741935483874,
 0.7712903225806451,
 0.7754301075268815,
 0.7801075268817202,
 0.7800537634408605,
 0.7804301075268815,
 0.7849462365591398,
 0.7829569892473122,
 0.7813440860215053,
 0.78172043010752

In [ ]:
train_loss_history

[array(0.49824136, dtype=float32),
 array(0.4849908, dtype=float32),
 array(0.38496912, dtype=float32),
 array(0.43650296, dtype=float32),
 array(0.42385706, dtype=float32),
 array(0.20700023, dtype=float32),
 array(0.23003152, dtype=float32),
 array(0.3109858, dtype=float32),
 array(0.4040987, dtype=float32),
 array(0.27382204, dtype=float32),
 array(0.17381284, dtype=float32),
 array(0.04715471, dtype=float32),
 array(0.2666063, dtype=float32),
 array(0.01195353, dtype=float32),
 array(0.00885608, dtype=float32),
 array(0.10372794, dtype=float32),
 array(0.02053358, dtype=float32),
 array(0.00463617, dtype=float32),
 array(0.00477684, dtype=float32),
 array(0.00363421, dtype=float32),
 array(0.00127381, dtype=float32),
 array(0.00591659, dtype=float32),
 array(0.00455574, dtype=float32),
 array(0.00082099, dtype=float32),
 array(0.13319547, dtype=float32),
 array(0.00306956, dtype=float32),
 array(0.00976817, dtype=float32),
 array(0.00063378, dtype=float32),
 array(0.00118564, dtype

In [ ]:
test_loss_history

[array(0.49824136, dtype=float32),
 array(0.4849908, dtype=float32),
 array(0.38496912, dtype=float32),
 array(0.43650296, dtype=float32),
 array(0.42385706, dtype=float32),
 array(0.20700023, dtype=float32),
 array(0.23003152, dtype=float32),
 array(0.3109858, dtype=float32),
 array(0.4040987, dtype=float32),
 array(0.27382204, dtype=float32),
 array(0.17381284, dtype=float32),
 array(0.04715471, dtype=float32),
 array(0.2666063, dtype=float32),
 array(0.01195353, dtype=float32),
 array(0.00885608, dtype=float32),
 array(0.10372794, dtype=float32),
 array(0.02053358, dtype=float32),
 array(0.00463617, dtype=float32),
 array(0.00477684, dtype=float32),
 array(0.00363421, dtype=float32),
 array(0.00127381, dtype=float32),
 array(0.00591659, dtype=float32),
 array(0.00455574, dtype=float32),
 array(0.00082099, dtype=float32),
 array(0.13319547, dtype=float32),
 array(0.00306956, dtype=float32),
 array(0.00976817, dtype=float32),
 array(0.00063378, dtype=float32),
 array(0.00118564, dtype